In [7]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk
import scipy.constants as cs

![Gilbert cell Topology](img/topology_Gilbert_Cell.png)

![diff_pair](img/diff_pair.png)

#### Technology data

In [8]:
n = lk('./nfet_03v3.mat')
p = lk('./pfet_03v3.mat')

#### Specifications

In [112]:
f_rf = 89 * cs.mega
f_lo = 100 * cs.mega
NF = 9 # [dB], baseband integrated from 1K to 100MHz
VDD = 3.3 # [V]
conversion_gain = 15 # [db], 20log10 (V_out / V_rf)
IIP2 = 40 # [dBm]
IIP3 = 8 # [dBm], using 2 tones at 2.4 GHz and 2.401 GHz
I0= 50e-6 # [uA] = I1

#### Design choices

In [113]:
l = 0.28
gm_id_dp_rf = np.array([12, 12])
gm_id_dp_lo = np.array([15, 15])

gm_rf = gm_id_dp_rf * I0
gm_lo = gm_id_dp_lo * I0/2

#### Sizing and benchmarking

![IIP3_req](img/IIP3_requirement_formula.png)

In [114]:
jd_rf = n.lookup('ID_W', GM_ID=gm_id_dp_rf, L=l)
jd_lo = n.lookup('ID_W', GM_ID=gm_id_dp_lo, L=l)

In [115]:
w_rf = I0 / jd_rf 
w_lo = I0 / jd_lo

cgg_w_rf = n.lookup('CGG_W', GM_ID=gm_id_dp_rf, L=l)
cgg_rf = w_rf*cgg_w_rf
ft_rf = gm_rf/cgg_rf/2/np.pi

cgg_w_lo = n.lookup('CGG_W', GM_ID=gm_id_dp_lo, L=l)
cgg_lo = w_lo*cgg_w_lo
ft_lo = gm_lo/cgg_lo/2/np.pi


for i in range(1):
    print (gm_id_dp_rf[i], gm_rf[i], I0, jd_rf[i], w_rf[i], ft_rf[i]/1e9)
    print (gm_id_dp_lo[i], gm_lo[i], I0, jd_lo[i], w_lo[i], ft_lo[i]/1e9)

12 0.0006000000000000001 5e-05 4.296084627443789e-06 11.63850443741153 7.6784943697789405
15 0.000375 5e-05 2.0974817168268422e-06 23.83811005306024 2.4458544133961806


In [116]:
RS = [1e3, 10e3, 100e3]
A_IIP3 = np.full_like(RS, 0)
for i in range(3):
    A_IIP3[i] = 2*I0 / 3 * np.sqrt(gm_rf[0] * (RS[i]+2/gm_rf[0])**3) # Watts
    A_IIP3[i] = 10 * np.log10(A_IIP3[i]/1e-3) # dBm
df = pd.DataFrame([RS, A_IIP3 ], ['RS', 'A_IIP3'], columns = ['option1', 'option2', 'option3'])
print(df) 

            option1       option2       option3
RS      1000.000000  10000.000000  100000.00000
A_IIP3    23.671875     30.993625      44.33315


#### Spice validation

In [64]:
%%writefile ./sizing_diffpair.spice
** differential pair sizing example

*.lib /foss/pdks/gf180mcuD/libs.tech/ngspice/sm141064.ngspice typical
*.inc /foss/pdks/gf180mcuD/libs.tech/ngspice/design.ngspice

* updated models that allow finger parameterization
.lib ../models_updated_2025.07.19/ngspice/sm141064.ngspice typical
.inc ../models_updated_2025.07.19/ngspice/design.ngspice

.param lx=0.28e-6 wx=146e-6 nfx=30, idx=1e-3

XM1a d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
XM1b d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
vg  g  0  1
vd  d  0  2
is  s  0  {2*idx}

.control
  op
  show
  save @m.xM1a.m0[gm]
  save @m.xM1a.m0[cgg]
  save @m.xM1a.m0[cgso]
  save @m.xM1a.m0[cgdo]

  print @m.xM1a.m0[gm]
  print @m.xM1a.m0[cgg]
  print @m.xM1a.m0[cgso]
  print @m.xM1a.m0[cgdo]
    
.endc
.end

Overwriting ./sizing_diffpair.spice


In [8]:
!PDK=gf180mcuD /foss/tools/bin/ngspice -b ./sizing_diffpair.spice

/bin/bash: line 1: /foss/tools/bin/ngspice: No such file or directory
